<a href="https://colab.research.google.com/github/honey-kolleri/Recomendation_System_Application/blob/main/Netflix_Movie_Recomendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Step:1 Install The Required Packages
!pip install pandas numpy matplotlib seaborn scikit-learn

In [2]:
#Step: 2 Import The Required Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
#Step:3 Load The Netflix_titles DataSet
df=pd.read_csv('netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [5]:
# Step 4: Combined The Key Columns with explicit string conversion
# We use .fillna('') on each column to ensure no 'nan' strings are created
df['Combine_Text'] = (
    df['title'].fillna('') + ' ' +
    df['director'].fillna('') + ' ' +
    df['cast'].fillna('') + ' ' +
    df['listed_in'].fillna('') + ' ' +
    df['description'].fillna('')
).str.lower() # Converting to lowercase helps with matching

# Double-check: Replace any remaining stray nulls in the final column
df['Combine_Text'] = df['Combine_Text'].fillna('')
# Optional: Verifying there are no nulls left
print(f"Number of nulls in Combine_Text: {df['Combine_Text'].isnull().sum()}")

Number of nulls in Combine_Text: 0


In [6]:
# Step 5: Converting the Combine_Text Column into Vectors
tfidf_object = TfidfVectorizer(stop_words='english', max_features=2000, ngram_range=(1, 2))

# Convert Text to Vectors (Keep as sparse matrix for speed/memory)
tfidf_matrix = tfidf_object.fit_transform(df['Combine_Text'])

# Skiping the creation of the tdidf_df (the DataFrame) to save RAM.
# The recommendation function will use tfidf_matrix instead.

In [7]:
#Step:6 Create Cosine_Similarity For Recommending The Similar Product
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [10]:
# Step 7: Recommendation Function
def recommend_movie(query, k=5):
    # Convert query text to vector
    query_vec = tfidf_object.transform([query])

    # Calculate similarity between query and all movies
    similarity = cosine_similarity(query_vec, tfidf_matrix).flatten()

    # Rank the movies and get top indices
    top_indices = similarity.argsort()[-k:][::-1]

    # Return the title, genre, and description of top matches
    result = df.iloc[top_indices][['title', 'type', 'release_year', 'listed_in', 'description']]
    return result
    # Usage
search_query = input('Enter a movie title, actor, or genre you like: ')
recommend_movie(search_query, 10)

Enter a movie title, actor, or genre you like: korean dramas


,title,type,release_year,listed_in,description
8347,The Homecoming,TV Show,2015,"International TV Shows, Korean TV Shows, Reali...","Panelists from the Korean show ""Abnormal Summi..."
1442,Korean Pork Belly Rhapsody,TV Show,2020,"Docuseries, International TV Shows",A love letter to pork belly — a perennial favo...
3928,Persona,TV Show,2019,"International TV Shows, Korean TV Shows, TV Dr...",An exploration of different personas in an ecl...
7670,Operation Chromite,Movie,2016,"Action & Adventure, Dramas, International Movies",To pave the way for a major amphibious invasio...
6079,Abnormal Summit,TV Show,2017,"International TV Shows, Korean TV Shows, Stand...","Led by a trio of Korean celebs, a multinationa..."
7041,I Am Sun Mu,Movie,2015,"Documentaries, International Movies",A North Korean propagandist who escaped to Seo...
4044,A Korean Odyssey,TV Show,2017,"International TV Shows, Korean TV Shows, Roman...",A self-serving mythical creature's bid for inv...
7101,마녀사냥,TV Show,2015,"International TV Shows, Korean TV Shows, Stand...",Four Korean celebrity men and guest stars of b...
245,Korean Cold Noodle Rhapsody,TV Show,2021,"Docuseries, International TV Shows","Refreshing and flavorful, naengmyeon is Korea'..."
2194,Unlucky Ploy,TV Show,2020,"International TV Shows, Romantic TV Shows, TV ...","In this remake of the Korean series, a man bec..."
